# Confirmed Cases and Deaths Code
### Latest update on 5/11/2022

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## FIPS code

In [2]:
# importing fips data. and rearranging the columns
fips = pd.read_excel("~/Documents/ra/HPC/HPC_datahub/Raw data/fips.xlsx")
fips = fips.iloc[:, [1, 3, 2, 4, 5]]

# Convert the 'integers' to integers in fips file.
for i in range(fips.shape[0]):
    if fips.iloc[i, 4] != 'unknown':
        fips.iloc[i, 4] = int(fips.iloc[i, 4])
        fips.iloc[i, 2] = int(fips.iloc[i, 2])

fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips = fips.reset_index()
fips = fips.iloc[0:3142, 1:]

# inspect the object
fips

,stfips,stname,ctyfips,ctyname,fips
0,1,Alabama,1,Autauga County,1001
1,1,Alabama,3,Baldwin County,1003
2,1,Alabama,5,Barbour County,1005
3,1,Alabama,7,Bibb County,1007
4,1,Alabama,9,Blount County,1009
...,...,...,...,...,...
3137,56,Wyoming,37,Sweetwater County,56037
3138,56,Wyoming,39,Teton County,56039
3139,56,Wyoming,41,Uinta County,56041
3140,56,Wyoming,43,Washakie County,56043


# Part I: County-Level Data and Dictionary

## Step 1: Update county-level data from CSSE

In [3]:
cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
cases.drop(cases.columns[[0,1,2, 3, 5, 6, 7, 8, 9, 10]], axis=1, inplace=True)

cases.rename(columns={"FIPS":"fips"}, inplace=True)

for col in cases.columns[1:]:
    cases.rename(columns={col: "confirmed_cases_" + datetime.strptime(col, "%m/%d/%y").strftime('%Y%m%d')}, inplace=True)

cases = pd.merge(fips, cases, on="fips", how="left")
#cases

In [4]:
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
deaths.drop(deaths.columns[[0,1,2, 3, 5, 6, 7, 8, 9, 10, 11]], axis=1, inplace=True)

deaths.rename(columns={"FIPS":"fips"}, inplace=True)

for col in deaths.columns[1:]:
    deaths.rename(columns={col: "deaths_" + datetime.strptime(col, "%m/%d/%y").strftime('%Y%m%d')}, inplace=True)

deaths = pd.merge(fips, deaths, on="fips", how="left")
#deaths

In [5]:
deaths = deaths.iloc[:, 5:]
csse = pd.concat([cases, deaths], axis=1)

In [6]:
# rename 'fips' column to 'scfips' to clarify it's state-county FIPS
csse.rename(columns = {'fips': 'scfips'}, inplace = True)
csse

,stfips,stname,ctyfips,ctyname,scfips,confirmed_cases_20200122,confirmed_cases_20200123,confirmed_cases_20200124,confirmed_cases_20200125,confirmed_cases_20200126,...,deaths_20220506,deaths_20220507,deaths_20220508,deaths_20220509,deaths_20220510,deaths_20220511,deaths_20220512,deaths_20220513,deaths_20220514,deaths_20220515
0,1,Alabama,1,Autauga County,1001,0,0,0,0,0,...,216,216,216,216,216,216,216,216,216,216
1,1,Alabama,3,Baldwin County,1003,0,0,0,0,0,...,680,680,680,680,680,680,680,681,681,681
2,1,Alabama,5,Barbour County,1005,0,0,0,0,0,...,98,98,98,98,98,98,98,98,98,98
3,1,Alabama,7,Bibb County,1007,0,0,0,0,0,...,104,104,104,104,104,104,105,105,105,105
4,1,Alabama,9,Blount County,1009,0,0,0,0,0,...,243,243,243,243,243,243,243,243,243,243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,Wyoming,37,Sweetwater County,56037,0,0,0,0,0,...,126,126,126,126,126,126,126,126,126,126
3138,56,Wyoming,39,Teton County,56039,0,0,0,0,0,...,16,16,16,16,16,16,16,16,16,16
3139,56,Wyoming,41,Uinta County,56041,0,0,0,0,0,...,39,39,39,39,39,39,39,39,39,39
3140,56,Wyoming,43,Washakie County,56043,0,0,0,0,0,...,44,44,44,44,44,44,44,44,44,44


## Step 2: Update CDC county-level cumulative provisional deaths data

In [7]:
cdcDeaths = pd.read_csv('https://data.cdc.gov/api/views/kn79-hsxy/rows.csv?accessType=DOWNLOAD')

In [8]:
cdcDeaths

,Date as of,Start Date,End Date,State,County name,FIPS County Code,Urban Rural Code,Deaths involving COVID-19,Deaths from All Causes,Footnote
0,05/11/2022,01/01/2020,05/07/2022,AK,Aleutians East Borough,2013,Noncore,NaN,16.0,One or more data cells have counts between 1-9...
1,05/11/2022,01/01/2020,05/07/2022,AK,Anchorage Municipality,2020,Medium metro,685.0,5941.0,NaN
2,05/11/2022,01/01/2020,05/07/2022,AK,Bethel Census Area,2050,Noncore,33.0,275.0,NaN
3,05/11/2022,01/01/2020,05/07/2022,AK,Denali Borough,2068,Noncore,NaN,14.0,One or more data cells have counts between 1-9...
4,05/11/2022,01/01/2020,05/07/2022,AK,Dillingham Census Area,2070,Noncore,NaN,81.0,One or more data cells have counts between 1-9...
...,...,...,...,...,...,...,...,...,...,...
3071,05/11/2022,01/01/2020,05/07/2022,WY,Sweetwater County,56037,Micropolitan,100.0,804.0,NaN
3072,05/11/2022,01/01/2020,05/07/2022,WY,Teton County,56039,Micropolitan,18.0,216.0,NaN
3073,05/11/2022,01/01/2020,05/07/2022,WY,Uinta County,56041,Micropolitan,20.0,344.0,NaN
3074,05/11/2022,01/01/2020,05/07/2022,WY,Washakie County,56043,Noncore,28.0,190.0,NaN


In [9]:
# rename variables
cdcDeaths.rename(columns = {'State': 'stabbr', 'County name': 'ctyname', 'FIPS County Code': 'scfips',\
                            'Urban Rural Code': 'urbanrural', 'Deaths involving COVID-19': 'death_covid',\
                            'Deaths from All Causes': 'death_all', 'Start Date': 'death_startdate',\
                            'End Date': 'death_enddate'}, \
                 inplace = True)

In [10]:
# format dates
cdcDeaths['death_startdate'] = pd.to_datetime(cdcDeaths['death_startdate'])
cdcDeaths['death_startdate'] = cdcDeaths['death_startdate'].dt.strftime('%Y%m%d')
cdcDeaths['death_enddate'] = pd.to_datetime(cdcDeaths['death_enddate'])
cdcDeaths['death_enddate'] = cdcDeaths['death_enddate'].dt.strftime('%Y%m%d')

In [11]:
# extract end date
endDate = list(set(cdcDeaths['death_enddate']))[0]

# add end date to variable names
cdcDeaths.rename(columns = {'death_covid': 'death_covid_' + endDate, 'death_all': 'death_all_' + endDate}, \
                 inplace = True)

In [12]:
# keep selected variables only
cdcDeaths = cdcDeaths[['stabbr', 'scfips', 'urbanrural', 'death_startdate', 'death_covid_' + endDate, \
                       'death_all_' + endDate]].copy()

In [13]:
cdcDeaths

,stabbr,scfips,urbanrural,death_startdate,death_covid_20220507,death_all_20220507
0,AK,2013,Noncore,20200101,NaN,16.0
1,AK,2020,Medium metro,20200101,685.0,5941.0
2,AK,2050,Noncore,20200101,33.0,275.0
3,AK,2068,Noncore,20200101,NaN,14.0
4,AK,2070,Noncore,20200101,NaN,81.0
...,...,...,...,...,...,...
3071,WY,56037,Micropolitan,20200101,100.0,804.0
3072,WY,56039,Micropolitan,20200101,18.0,216.0
3073,WY,56041,Micropolitan,20200101,20.0,344.0
3074,WY,56043,Noncore,20200101,28.0,190.0


## Step 3: Merge county-level data

In [14]:
# specify fips to be scfips
fips.rename(columns = {'fips': 'scfips'}, inplace = True)

In [15]:
# merge fips with csse
csse = fips.merge(csse, on = ['stfips', 'stname', 'ctyfips', 'ctyname', 'scfips'], how = 'outer')

In [17]:
# merge with cdc deaths
final = csse.merge(cdcDeaths, on = 'scfips', how = 'left')

In [18]:
final.columns

Index(['stfips', 'stname', 'ctyfips', 'ctyname', 'scfips',
       'confirmed_cases_20200122', 'confirmed_cases_20200123',
       'confirmed_cases_20200124', 'confirmed_cases_20200125',
       'confirmed_cases_20200126',
       ...
       'deaths_20220511', 'deaths_20220512', 'deaths_20220513',
       'deaths_20220514', 'deaths_20220515', 'stabbr', 'urbanrural',
       'death_startdate', 'death_covid_20220507', 'death_all_20220507'],
      dtype='object', length=1700)

In [19]:
final = final[['stfips', 'stabbr', 'stname', 'ctyfips', 'ctyname', 'scfips', 'urbanrural'] \
              + [x for x in list(final.columns)[5:] if x not in ['stabbr', 'urbanrural']]]

In [20]:
final['stfips'] = final['stfips'].astype('int')
final['scfips'] = final['scfips'].astype('int')
final

,stfips,stabbr,stname,ctyfips,ctyname,scfips,urbanrural,confirmed_cases_20200122,confirmed_cases_20200123,confirmed_cases_20200124,...,deaths_20220509,deaths_20220510,deaths_20220511,deaths_20220512,deaths_20220513,deaths_20220514,deaths_20220515,death_startdate,death_covid_20220507,death_all_20220507
0,1,AL,Alabama,1,Autauga County,1001,Medium metro,0,0,0,...,216,216,216,216,216,216,216,20200101,171.0,1282.0
1,1,AL,Alabama,3,Baldwin County,1003,Small metro,0,0,0,...,680,680,680,680,681,681,681,20200101,597.0,5766.0
2,1,AL,Alabama,5,Barbour County,1005,Noncore,0,0,0,...,98,98,98,98,98,98,98,20200101,75.0,666.0
3,1,AL,Alabama,7,Bibb County,1007,Large fringe metro,0,0,0,...,104,104,104,105,105,105,105,20200101,45.0,479.0
4,1,AL,Alabama,9,Blount County,1009,Large fringe metro,0,0,0,...,243,243,243,243,243,243,243,20200101,88.0,1236.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56,WY,Wyoming,37,Sweetwater County,56037,Micropolitan,0,0,0,...,126,126,126,126,126,126,126,20200101,100.0,804.0
3138,56,WY,Wyoming,39,Teton County,56039,Micropolitan,0,0,0,...,16,16,16,16,16,16,16,20200101,18.0,216.0
3139,56,WY,Wyoming,41,Uinta County,56041,Micropolitan,0,0,0,...,39,39,39,39,39,39,39,20200101,20.0,344.0
3140,56,WY,Wyoming,43,Washakie County,56043,Noncore,0,0,0,...,44,44,44,44,44,44,44,20200101,28.0,190.0


In [21]:
# output
final.to_csv("../Pandemic/county_casesAndDeaths.csv", index = False)

## Step 4: Update county-level dictionary

In [22]:
from get_dict_function import get_dict 

In [23]:
cDict = pd.read_csv('../Pandemic/county_data_dictionary.csv')

In [24]:
tDict = cDict[cDict['field'] == 'cases and deaths']

In [25]:
tDict.variable_name

0                       stfips
1                       stabbr
2                       stname
3                      ctyfips
4                      ctyname
5                       scfips
6                   urbanrural
7     confirmed_cases_yyyymmdd
8              deaths_yyyymmdd
9              death_startdate
10          death_all_20220430
11        death_covid_20220430
Name: variable_name, dtype: object

In [25]:
'''
# move urbanrural variable to after scfips on 4/11/2022: run once
tDict1 = tDict.iloc[:6, :]
tDict2 = tDict.iloc[9, :]
tDict3 = tDict.iloc[6:9, :]
tDict4 = tDict.iloc[10:, :]
tDict = tDict1.append(tDict2, ignore_index = True)
tDict = tDict.append(tDict3, ignore_index = True)
tDict = tDict.append(tDict4, ignore_index = True)
tDict
'''

'\n# move urbanrural variable to after scfips on 4/11/2022: run once\ntDict1 = tDict.iloc[:6, :]\ntDict2 = tDict.iloc[9, :]\ntDict3 = tDict.iloc[6:9, :]\ntDict4 = tDict.iloc[10:, :]\ntDict = tDict1.append(tDict2, ignore_index = True)\ntDict = tDict.append(tDict3, ignore_index = True)\ntDict = tDict.append(tDict4, ignore_index = True)\ntDict\n'

In [26]:
tDict.loc[10, 'variable_name'] = 'death_all_' + endDate
tDict.loc[11, 'variable_name'] = 'death_covid_' + endDate

/Users/xywu/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [27]:
tDict.variable_name

0                       stfips
1                       stabbr
2                       stname
3                      ctyfips
4                      ctyname
5                       scfips
6                   urbanrural
7     confirmed_cases_yyyymmdd
8              deaths_yyyymmdd
9              death_startdate
10          death_all_20220507
11        death_covid_20220507
Name: variable_name, dtype: object

In [28]:
tDict = get_dict(final, tDict)

0
Single column for this item...
1
Single column for this item...
2
Single column for this item...
3
Single column for this item...
4
Single column for this item...
5
Single column for this item...
6
Single column for this item...
7
Multiple columns for this item...
8
Multiple columns for this item...
9
Single column for this item...
10
Single column for this item...
11
Single column for this item...


In [29]:
tDict

,variable_name,label,file,field,source,link,start_column,end_column,start_date,end_date,filename_in_release4
0,stfips,State name,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,1,1,-1,-1,casesAndDeaths.csv
1,stabbr,State abbreviation,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,2,2,-1,-1,casesAndDeaths.csv
2,stname,State FIPS code,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,3,3,-1,-1,casesAndDeaths.csv
3,ctyfips,County name,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,4,4,-1,-1,casesAndDeaths.csv
4,ctyname,County FIPS code,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,5,5,-1,-1,casesAndDeaths.csv
5,scfips,FIPS code of state-county,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,6,6,-1,-1,casesAndDeaths.csv
6,urbanrural,NCHS urban-rural classification,county_casesAndDeaths.csv,cases and deaths,CDC,https://data.cdc.gov/NCHS/Provisional-COVID-19...,7,7,-1,-1,NaN
7,confirmed_cases_yyyymmdd,No of confirmed cases on yyyymmdd,county_casesAndDeaths.csv,cases and deaths,CSSE,https://github.com/CSSEGISandData/COVID-19,8,852,20200122,20220515,casesAndDeaths.csv
8,deaths_yyyymmdd,No of deaths on yyyymmdd,county_casesAndDeaths.csv,cases and deaths,CSSE,https://github.com/CSSEGISandData/COVID-19,853,1697,20200122,20220515,casesAndDeaths.csv
9,death_startdate,First date of data period,county_casesAndDeaths.csv,cases and deaths,CDC,https://data.cdc.gov/NCHS/Provisional-COVID-19...,1698,1698,-1,-1,NaN


In [30]:
cDict[cDict['field'] == 'cases and deaths'] = tDict.copy()

In [31]:
cDict

,variable_name,label,file,field,source,link,start_column,end_column,start_date,end_date,filename_in_release4
0,stfips,State name,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,1,1,-1,-1,casesAndDeaths.csv
1,stabbr,State abbreviation,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,2,2,-1,-1,casesAndDeaths.csv
2,stname,State FIPS code,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,3,3,-1,-1,casesAndDeaths.csv
3,ctyfips,County name,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,4,4,-1,-1,casesAndDeaths.csv
4,ctyname,County FIPS code,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,5,5,-1,-1,casesAndDeaths.csv
5,scfips,FIPS code of state-county,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-f...,6,6,-1,-1,casesAndDeaths.csv
6,urbanrural,NCHS urban-rural classification,county_casesAndDeaths.csv,cases and deaths,CDC,https://data.cdc.gov/NCHS/Provisional-COVID-19...,7,7,-1,-1,NaN
7,confirmed_cases_yyyymmdd,No of confirmed cases on yyyymmdd,county_casesAndDeaths.csv,cases and deaths,CSSE,https://github.com/CSSEGISandData/COVID-19,8,852,20200122,20220515,casesAndDeaths.csv
8,deaths_yyyymmdd,No of deaths on yyyymmdd,county_casesAndDeaths.csv,cases and deaths,CSSE,https://github.com/CSSEGISandData/COVID-19,853,1697,20200122,20220515,casesAndDeaths.csv
9,death_startdate,First date of data period,county_casesAndDeaths.csv,cases and deaths,CDC,https://data.cdc.gov/NCHS/Provisional-COVID-19...,1698,1698,-1,-1,NaN


In [32]:
# output
cDict.to_csv("../Pandemic/county_data_dictionary.csv", index = False)

# Part II: State-Level Data and Dictionary

## Step 1: Update CDC provisional deaths data by place of death and state

In [33]:
deaths = pd.read_csv('https://data.cdc.gov/api/views/uggs-hy5q/rows.csv?accessType=DOWNLOAD')
deaths.columns

Index(['Data As Of', 'Start Date', 'End Date', 'Group', 'Year', 'Month',
       'State', 'Place of Death', 'COVID-19 Deaths', 'Total Deaths',
       'Pneumonia Deaths', 'Pneumonia and COVID-19 Deaths', 'Influenza Deaths',
       'Pneumonia, Influenza, or COVID-19 Deaths', 'Footnote'],
      dtype='object')

In [34]:
footnote = list(set(deaths.Footnote))[1]
footnote

'One or more data cells have counts between 1-9 and have been suppressed in accordance with NCHS confidentiality standards.'

In [35]:
# selected by month, state-level, and all-aces-of-deaths rows
deaths = deaths[deaths['Group'] == 'By Month'].copy()
deaths = deaths[deaths['Place of Death'] == 'Total - All Places of Death'].copy()
deaths = deaths[deaths['State'] != 'United States'].copy()
deaths.reset_index(drop = True, inplace = True)

In [36]:
# select columns
deaths = deaths[['State', 'Year', 'Month', 'COVID-19 Deaths', 'Total Deaths',\
                 'Pneumonia Deaths', 'Pneumonia and COVID-19 Deaths', 'Influenza Deaths',\
                 'Pneumonia, Influenza, or COVID-19 Deaths']].copy()

In [37]:
# generate year-month variable
deaths.loc[:, 'Year'] = deaths['Year'].astype('int')
deaths.loc[:, 'Month'] = deaths['Month'].astype('int')
# turn 1- or 2-digit month in integer into 2-digit mongh string
deaths.loc[:, 'Month'] = [str(x).zfill(2) for x in list(deaths.Month)]
# concatenate year and month info
deaths['year_month'] = deaths['Year'].astype('str') + deaths['Month']
deaths.drop(columns = ['Year', 'Month'], inplace = True)

In [38]:
# rename variables before reshape
deaths.rename(columns = {'Total Deaths': 'death_all_',\
                         'COVID-19 Deaths': 'death_covid_',\
                         'Pneumonia Deaths': 'death_pneu_',\
                         'Pneumonia and COVID-19 Deaths': 'death_cp_',\
                         'Influenza Deaths': 'death_flu_',\
                         'Pneumonia, Influenza, or COVID-19 Deaths': 'death_cpf_'},\
              inplace = True)

In [39]:
# get summary statistics before reshape to see if any suspicious records
smry = deaths[['death_all_', 'death_covid_', 'death_pneu_', 'death_cp_',\
               'death_flu_', 'death_cpf_']].describe().round(decimals = 2)
# check if any null values in all-cause death counts
print(smry['death_all_'].isnull().values.any()) # false, good to proceed
# inspect summary statistics manually
#smry.to_csv('~/Documents/ra/HPC/HPC_datahub/release6/cdc_provisional_deaths_state_summary.csv', index = True)
smry

False


,death_all_,death_covid_,death_pneu_,death_cp_,death_flu_,death_cpf_
count,1536.00,1452.00,1512.00,1399.00,809.00,1515.00
mean,5229.23,690.49,594.32,370.50,11.48,919.05
std,5602.44,1321.18,920.86,751.33,37.26,1480.55
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,1354.00,70.75,127.00,37.00,0.00,178.00
50%,3767.00,263.50,330.00,133.00,0.00,463.00
75%,6589.25,765.00,666.50,402.00,11.00,1046.50
max,48158.00,20060.00,13801.00,12191.00,746.00,21681.00


In [40]:
# get unique items as a prepare for dictionary
itms = list(deaths.columns)
itms.remove('State')
itms.remove('year_month')
itms = ['cdc_' + x + 'yyyymm' for x in itms]

In [41]:
# reshape from long form to wide form
deaths = deaths.pivot(index = 'State', columns = 'year_month')
deaths.columns = [x[0] + x[1] for x in list(deaths.columns)]
deaths.reset_index(drop = False, inplace = True)
deaths.rename(columns = {'State': 'stname'}, inplace = True)

In [42]:
# merge with fips code
fips = pd.read_csv('~/Documents/GitHub/COVID_DataHub/FIPS/state_territory_fips.csv')
tdf = fips.merge(deaths, on = 'stname', how = 'outer')
tdf.iloc[:, :3]
# combine NYC into NY state
tdf.loc[57, 'stname'] = 'New York'
tdf = tdf.groupby(by = ['stfips', 'stabbr', 'stname'], as_index = False).sum()

In [43]:
# add 'cdc' prefix to column names
tdf.columns = list(tdf.columns)[:3] + ['cdc_' + x for x in list(tdf.columns)[3:]]

In [44]:
tdf

,stfips,stabbr,stname,cdc_death_covid_202001,cdc_death_covid_202002,cdc_death_covid_202003,cdc_death_covid_202004,cdc_death_covid_202005,cdc_death_covid_202006,cdc_death_covid_202007,...,cdc_death_cpf_202108,cdc_death_cpf_202109,cdc_death_cpf_202110,cdc_death_cpf_202111,cdc_death_cpf_202112,cdc_death_cpf_202201,cdc_death_cpf_202202,cdc_death_cpf_202203,cdc_death_cpf_202204,cdc_death_cpf_202205
0,1.0,AL,Alabama,0.0,0.0,48.0,339.0,430.0,399.0,879.0,...,1903.0,2199.0,1094.0,529.0,575.0,1511.0,1470.0,581.0,201.0,13.0
1,2.0,AK,Alaska,0.0,0.0,0.0,0.0,0.0,0.0,12.0,...,122.0,206.0,234.0,132.0,81.0,90.0,89.0,38.0,15.0,0.0
2,4.0,AZ,Arizona,0.0,0.0,64.0,405.0,507.0,899.0,2102.0,...,1074.0,1473.0,1436.0,1925.0,2512.0,2449.0,1614.0,712.0,283.0,20.0
3,5.0,AR,Arkansas,0.0,0.0,10.0,61.0,78.0,169.0,274.0,...,1118.0,908.0,580.0,407.0,575.0,1018.0,925.0,352.0,164.0,13.0
4,6.0,CA,California,0.0,0.0,279.0,2021.0,2238.0,2065.0,3890.0,...,4694.0,5503.0,4066.0,3790.0,3987.0,8359.0,6182.0,2653.0,1115.0,43.0
5,8.0,CO,Colorado,0.0,0.0,107.0,880.0,465.0,184.0,152.0,...,481.0,676.0,971.0,1384.0,1245.0,1309.0,782.0,386.0,146.0,10.0
6,9.0,CT,Connecticut,0.0,0.0,126.0,2457.0,1535.0,285.0,77.0,...,262.0,300.0,307.0,328.0,548.0,1149.0,488.0,202.0,154.0,0.0
7,10.0,DE,Delaware,0.0,0.0,0.0,189.0,276.0,73.0,35.0,...,86.0,164.0,156.0,124.0,225.0,383.0,192.0,101.0,41.0,0.0
8,11.0,DC,District of Columbia,0.0,0.0,15.0,262.0,272.0,116.0,43.0,...,71.0,61.0,67.0,46.0,100.0,224.0,96.0,52.0,22.0,0.0
9,12.0,FL,Florida,0.0,0.0,191.0,1311.0,1131.0,1083.0,4980.0,...,11485.0,9385.0,3733.0,1872.0,1977.0,5829.0,4444.0,2142.0,1002.0,71.0


## Step 2: Update state-level data file

In [45]:
# read existing data
slData = pd.read_csv('~/Documents/GitHub/COVID_DataHub/Pandemic/state_level_data.csv')

# read state-level dictionary
slDict = pd.read_csv('~/Documents/GitHub/COVID_DataHub/Pandemic/state_data_dictionary.csv')

In [46]:
# find indices of state-level cases & deaths variables
rows = slDict.index[slDict['link'] == 'https://data.cdc.gov/NCHS/Provisional-COVID-19-Deaths-by-Place-of-Death-and-/uggs-hy5q'].tolist()
# get the range of indices
cols = []
for i in range(len(rows)):
    cols = cols + list(range(slDict.loc[rows[i],'start_column'] - 1,\
                             slDict.loc[rows[i], 'end_column']))
# double-check if consecutive
print(cols[0])
print(cols[-1])
cols == list(range(cols[0], cols[-1]))
# if true, good to proceed: insert this deaths data to assigned location

# CAUTION: since the column number starts from 1, which is different from python's starting column index as 0,
#          need to minus one for the starting column to get the real column index. The end column index is good
#          as python doesn't include the higher bound anyway.

108
275


False

In [48]:
#cols

In [49]:
# split state-level data into two trunks: 
# (1) before the start of previous provisional death data
# (2) after the end of cases & deaths data
df0 = slData.iloc[:, :(cols[0])]
df2 = slData.iloc[:, (cols[-1] + 1):]

In [50]:
df0.columns

Index(['stname', 'stabbr', 'stfips', 'apm_cases_total_2018',
       'apm_cases_hispanic_2018', 'apm_cases_nhwhite_2018',
       'apm_cases_nhblack_2018', 'apm_cases_nhasian_2018',
       'apm_cases_totalDeathBy_20200609', 'apm_cases_totalDeathBy_20200623',
       ...
       'apm_cases_asianDeath_20201027', 'apm_cases_asianDeath_20201110',
       'apm_cases_asianDeath_20201124', 'apm_cases_asianDeath_20201208',
       'apm_cases_asianDeath_20201222', 'apm_cases_asianDeath_20210105',
       'apm_cases_asianDeath_20210119', 'apm_cases_asianDeath_20210202',
       'apm_cases_asianDeath_20210216', 'apm_cases_asianDeath_20210302'],
      dtype='object', length=108)

In [51]:
slData.iloc[:, cols[0]:(cols[-1] + 1)].columns

Index(['cdc_death_covid_202001', 'cdc_death_covid_202002',
       'cdc_death_covid_202003', 'cdc_death_covid_202004',
       'cdc_death_covid_202005', 'cdc_death_covid_202006',
       'cdc_death_covid_202007', 'cdc_death_covid_202008',
       'cdc_death_covid_202009', 'cdc_death_covid_202010',
       ...
       'cdc_death_cpf_202107', 'cdc_death_cpf_202108', 'cdc_death_cpf_202109',
       'cdc_death_cpf_202110', 'cdc_death_cpf_202111', 'cdc_death_cpf_202112',
       'cdc_death_cpf_202201', 'cdc_death_cpf_202202', 'cdc_death_cpf_202203',
       'cdc_death_cpf_202204'],
      dtype='object', length=168)

In [52]:
df2.columns

Index(['medical_covidtracking_positive_20200113',
       'medical_covidtracking_positive_20200114',
       'medical_covidtracking_positive_20200115',
       'medical_covidtracking_positive_20200116',
       'medical_covidtracking_positive_20200117',
       'medical_covidtracking_positive_20200118',
       'medical_covidtracking_positive_20200119',
       'medical_covidtracking_positive_20200120',
       'medical_covidtracking_positive_20200121',
       'medical_covidtracking_positive_20200122',
       ...
       'apm_vac_latino_20210503', 'apm_vac_latino_20210517',
       'apm_vac_indig_20210405', 'apm_vac_indig_20210419',
       'apm_vac_indig_20210503', 'apm_vac_indig_20210517',
       'apm_vac_other_20210405', 'apm_vac_other_20210419',
       'apm_vac_other_20210503', 'apm_vac_other_20210517'],
      dtype='object', length=11750)

In [53]:
# merge provisional deaths data with the first half
df1 = df0.merge(tdf, on = ['stfips', 'stabbr', 'stname'], how = 'left')
# concatenate df0 and df1 on columns
fnl = pd.concat([df1, df2], axis = 1)

In [54]:
# output updated data
fnl.to_csv('~/Documents/GitHub/COVID_DataHub/Pandemic/state_level_data.csv', index = False)

## Step 3: Update state-level dictionary

In [55]:
# update dictionary
fdct = get_dict(fnl, slDict)

0
Single column for this item...
1
Single column for this item...
2
Single column for this item...
3
Single column for this item...
4
Single column for this item...
5
Single column for this item...
6
Single column for this item...
7
Single column for this item...
8
Multiple columns for this item...
9
Multiple columns for this item...
10
Multiple columns for this item...
11
Multiple columns for this item...
12
Multiple columns for this item...
13
Multiple columns for this item...
14
Multiple columns for this item...
15
Multiple columns for this item...
16
Multiple columns for this item...
17
Multiple columns for this item...
18
Multiple columns for this item...
19
Multiple columns for this item...
20
Multiple columns for this item...
21
Multiple columns for this item...
22
Multiple columns for this item...
23
Multiple columns for this item...
24
Multiple columns for this item...
25
Multiple columns for this item...
26
Multiple columns for this item...
27
Multiple columns for this item..

In [56]:
fdct

,variable_name,label,field,source,link,start_column,end_column,start_date,end_date,filename_in_release4,footnote
0,stname,State name,state identifier,Census,https://www2.census.gov/geo/docs/reference/sta...,1,1,-1,-1,NaN,NaN
1,stabbr,State abbreviation,state identifier,Census,https://www2.census.gov/geo/docs/reference/sta...,2,2,-1,-1,NaN,NaN
2,stfips,State FIPS code,state identifier,Census,https://www2.census.gov/geo/docs/reference/sta...,3,3,-1,-1,NaN,NaN
3,apm_cases_total_2018,Total population in 2018,cases and deaths,APM,https://www.apmresearchlab.org/covid/deaths-by...,4,4,-1,-1,ApmColorOfCoronavirus.csv,NaN
4,apm_cases_hispanic_2018,Hispanic pooulation in 2018,cases and deaths,APM,https://www.apmresearchlab.org/covid/deaths-by...,5,5,-1,-1,ApmColorOfCoronavirus.csv,NaN
...,...,...,...,...,...,...,...,...,...,...,...
135,apm_vac_white_yyyymmdd,Number of White persons vaccinated,vaccination,APM,https://www.apmresearchlab.org/covid/vaccines-...,12013,12016,20210405,20210517,NaN,NaN
136,apm_vac_asian_yyyymmdd,Number of Asian persons vaccinated,vaccination,APM,https://www.apmresearchlab.org/covid/vaccines-...,12017,12020,20210405,20210517,NaN,NaN
137,apm_vac_latino_yyyymmdd,Number of Latino persons vaccinated,vaccination,APM,https://www.apmresearchlab.org/covid/vaccines-...,12021,12024,20210405,20210517,NaN,NaN
138,apm_vac_indig_yyyymmdd,Number of Indigenous persons vaccinated,vaccination,APM,https://www.apmresearchlab.org/covid/vaccines-...,12025,12028,20210405,20210517,NaN,NaN


In [57]:
# output updated dictionary
fdct.to_csv('~/Documents/GitHub/COVID_DataHub/Pandemic/state_data_dictionary.csv', index = False)